# **Model training and Generating**

**Download Required Message**

In [ ]:
!pip install accelerate -U
!pip install transformers[torch]

import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel

**Check for whether the GPU acclerated is available**

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


**Define the Generation Logic**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path, use_safetensors = True)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length, histories = [], testOrNot = True):
    model_path = "/content/drive/MyDrive/CuringBot/CuringBot_Model"

    # acclerate it with GPU
    model = load_model(model_path).to(device)
    tokenizer = load_tokenizer(model_path)

    real_sequence = ""

    # if in the real chatbot, we shall append the history
    if not testOrNot:
      if len(histories) < 4:
        for i in range(len(histories)):
          real_sequence += histories[i]
      else:
        for i in range(len(histories) - 4, len(histories)):
          real_sequence += histories[i]

    # the new maxlen based on the appended history
    current_max_len = len(real_sequence)
    max_length += current_max_len

    # the current round of talking
    new_sequence = f'User: <s> {sequence} </s> '
    tohistoryuser = f'User: <s> {sequence} </s> '
    real_sequence += new_sequence

    real_sequence=real_sequence.replace(" .",".").replace(" ,","").replace(" ?","").replace(" !","").replace(" ......","......")

    if not testOrNot:
      histories.append(tohistoryuser)

    # acclerate it with GPU
    ids = tokenizer.encode(real_sequence, return_tensors='pt').to(device)
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
    )

    raw_respond = (tokenizer.decode(final_outputs[0], skip_special_tokens=True))
    raw_respond_generated = raw_respond.replace(real_sequence, "")
    # print(real_sequence)
    # print(raw_respond)
    output = raw_respond_generated.split("User:")[0]


    if not testOrNot:
      histories.append(f"Assistant: <s> {output}")

    output = output.replace(" <s> ", "").replace(" </s>", "")
    return output

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Chatbot Functionality Test**

In [ ]:
sequence = "User: <s> Thank you , Alex . </s> <s> Your support means a lot to me . </s> <s> I think for now , I'll explore painting as my new hobby and work on shifting my mindset . </s> <s> If any further challenges arise , I may reach out to you again . </s> <s> I truly appreciate your help . </s> "
max_len = 256
print(len(sequence))
print(generate_text(sequence, max_len))

289

Assistant:You're welcome, Charlie. I'm here for you whenever you need support. Take care of yourself and remember to be kind to yourself.



In [ ]:
histories = []
print("Please Enter the message within prompt: ")
IndicationMethod = "123"
while True:
  print("User: ")
  sequence = input()
  max_len = 128
  if sequence == "Bye":
    break
  print(generate_text(sequence, max_len, histories, False))

Please Enter the message within prompt: 
User: 
Hi, I am so mentally broken

Assistant:Hello Charlie, thank you for sharing your feelings with me. I can sense that you're going through a difficult time. Can you tell me more about what happened?

User: 
I am failing the exam and the course, so I am so worried and upset

Assistant:I can understand why you would feel that way. It sounds like you're going through a challenging time. Have you had a chance to talk to your parents about how you're feeling?

User: 
Will talk to my parents really gonna help?

Assistant:That's a good point, Charlie. It's important to express your concerns to your parents and let them know how their actions are affecting you. What specifically is causing you to feel anxious about the exam?

User: 
I don't do well in midterm, and my cumulative score is low now

Assistant:It sounds like the math test is affecting your performance. Have you tried addressing this issue with your parents?

User: 
No, I haven't. Do you

KeyboardInterrupt: Interrupted by user

# **Auto Evaluation Section**

**Load the Test Dataset**

In [ ]:
# Set the environment to UTF-8
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install lmppl
!pip install nltk
!pip install pandas
!pip install numpy

In [ ]:
import pandas as pd

# Read the contents of the text file
with open("/content/drive/MyDrive/CuringBot/test_set_42020.txt", 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Initialize lists to store user and assistant utterances
user_utterances = []
assistant_utterances = []

# Iterate through the lines and separate user and assistant utterances
for line in lines:
    if line.startswith('User:'):
        # Remove <s> and </s> tags and strip whitespace
        user_utterance = line
        user_utterances.append(user_utterance)
    elif line.startswith('Assistant:'):
        # Remove <s> and </s> tags and strip whitespace
        assistant_utterance = line
        assistant_utterances.append(assistant_utterance)

print(user_utterances[:5])
print(assistant_utterances[:5])

["User: <s> Hi Alex , I hope you're doing well . </s> <s> I've been feeling really stressed lately and I'm experiencing a deep sense of isolation . </s> <s> I thought maybe talking to you could help . </s>\n", "User: <s> Thank you , Alex . </s> <s> Lately , work has been overwhelming and it feels like there's never enough time to take care of myself . </s> <s> I've been working long hours and it's been impacting my mental and physical well-being . </s> <s> It's difficult to find a balance , which leaves me feeling isolated from my own needs and from others . </s>\n", "User: <s> Well , I've been contemplating picking up a new hobby . </s> <s> My friend recently started painting and she seems to find solace in it . </s> <s> I've always been drawn to art and expressing myself creatively , so painting might be a good outlet for me . </s> <s> But I don't know where to start , and I'm worried I won't be good enough . </s>\n", "User: <s> You're right , Alex . </s> <s> I shouldn't put too much

In [ ]:
import numpy as np

# Load a pre-trained model (replace with your desired model)
model = load_model("/content/drive/MyDrive/CuringBot/CuringBot_Model/").to(device)
tokenizer = load_tokenizer("/content/drive/MyDrive/CuringBot/CuringBot_Model/")

inputs = user_utterances[0:1000]
perplexities = []
for input in inputs:
  ids = tokenizer(f'{input}', return_tensors='pt').to(device)
  with torch.no_grad():
      losses = model(input_ids = ids["input_ids"], labels = ids["input_ids"]).loss

  # Compute perplexity
  perplexity = torch.exp(losses)
  perplexities.append(perplexity)

# Calculate perplexity
perplexity = sum(perplexities) / len(perplexities)
print(f"Perplexity: {perplexity}")

Perplexity: 7.391539096832275


**Calculate the BLEU Score**

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

# Define your reference and candidate sentences (model output)
inputs = user_utterances[0:100]
references = assistant_utterances[0:100]
candidate = []
bleu_scores = []

itr=0;
for i in inputs:
  outputs = generate_text(i, 256)
  candidate.append(outputs)
  itr+=1


for input_seq, output_seq, reference in zip(inputs, candidate, references):
  output_tokens = output_seq.split()
  reference_tokens = reference.split()
  bleu_score = sentence_bleu([reference_tokens], output_tokens)
  bleu_scores.append(bleu_score)


# Print the BLEU score
average_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("BLEU Score:", average_bleu_score)

BLEU Score: 0.01802369870461459


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

**Calculate the BERT Score**

In [ ]:
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertModel
from bert_score import BERTScorer
# Example texts
inputs = user_utterances[0:100]
references = assistant_utterances[0:100]

BERTPrecision = []
BERTRecall = []
BERTF1 = []

for i in range(len(inputs)):
  reference = references[i]
  candidate = generate_text(inputs[i], 256)
  # BERTScore calculation
  scorer = BERTScorer(model_type='bert-base-uncased')
  P, R, F1 = scorer.score([candidate], [reference])
  BERTPrecision.append(P.mean())
  BERTRecall.append(R.mean())
  BERTF1.append(F1.mean())

# Print out the average overall:
print("average Bert Precision: ", np.sum(BERTPrecision) / len(BERTPrecision))
print("average Bert Recall: ", np.sum(BERTRecall) / len(BERTRecall))
print("average Bert F1: ", np.sum(BERTF1) / len(BERTF1))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

average Bert Precision:  0.624126205444336
average Bert Recall:  0.4502383804321289
average Bert F1:  0.5224925231933594
